<a href="https://colab.research.google.com/github/Axmeddddd/VGsim_Params_Guesser/blob/main/VGsim_params_guesser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Some basic imports**

In [ ]:
import pandas as pd
import torch
import torch.nn as nn

**Creating a neural network architecture with 3 fully connected layers and hyperbolic tangent activation function**

In [ ]:
class Params_Guesser(nn.Module):

  def __init__(self):
    super(Params_Predictor, self).__init__()
    self.fc1 = nn.Linear(29, 841)
    self.fc2 = nn.Linear(841, 841)
    self.fc3 = nn.Linear(841, 2)
    self.act = nn.Tanh()

  def forward(self, input):
    output = self.fc1(input)
    output = self.act(output)
    output = self.fc2(output)
    output = self.act(output)
    output = self.fc3(output)
    return output

**Converting csv file into pd.Dataframe and then normalize all the values in that table**

In [ ]:
df = pd.read_csv('samples.csv')

df = (df-df.min())/(df.max()-df.min())

**Training the neural network by SGD algorithm with learning rate of 0.005, 100 epochs and a batch size equal to one pd.Series length**

In [ ]:
model = Params_Guesser()
optimizer_model = torch.optim.SGD(model.parameters(), lr=0.005)
criterion_model = nn.MSELoss()

for epoch in range(100):
  total_loss = []
  for batch in range(4074):
    input = torch.tensor(df.iloc[batch, : 29], dtype=torch.float).view(1, 29)
    target = torch.tensor(df.iloc[batch, -2 : ], dtype=torch.float).view(1, 2)
    optimizer_model.zero_grad()                                                 # nulling the gradient on each iteration
    outputs = model(input)
    loss = criterion_model(outputs, target)
    total_loss.append(loss.item())
    loss.backward()                                                             # gradient calculating
    optimizer_model.step()                                                      # changing of weights
  total_loss = sum(total_loss) / 4074
  print(total_loss)

0.013973150977881576
0.014896380411434396
0.014626329139642288
0.015167412551672763
0.01563398867926582
0.015996430963484083
0.01630884012104358
0.016593796470374338
0.016857576224546075
0.01710134990007989
0.01732495538886138
0.017527954060938294
0.017709885806883405
0.017870279611308798
0.018008641378662444
0.018124431397114622
0.0182170867398214
0.01828608742435793
0.018331093052652726
0.018352131750210037
0.018349844794030203
0.01832572399651664
0.018282294399406207
0.018223135125761156
0.018152716230052497
0.018076001074552178
0.01799793886120085
0.017922908755705964
0.01785428037921532
0.0177941686745848
0.017743401044154308
0.017701667176300225
0.017667765980972562
0.01763991618864639
0.017616040940269626
0.017594011781551504
0.017571835183170587
0.017547777903420204
0.017520441520637503
0.017488796793762484
0.017452180977830967
0.01741025802468999
0.017362991598207034
0.01731058297514258
0.017253420593323573
0.01719202879559056
0.01712704051926017
0.017059166475528877
0.0169891

**We've got trained model with final absolute error = 0.017563925102973384 and a relative error about 1-2%. Very good! Then we save weights of our network into .ptr file**

In [ ]:
torch.save(model.state_dict(), "params_predictor.ptr")